In [16]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.models import load_model, save_model
from keras.utils import get_file, to_categorical
from imblearn.over_sampling import SMOTE
from collections import Counter
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

In [17]:
pd.set_option('display.max_columns', 100)

df = pd.read_csv('CSV\KDDCUP99\kdd-with-columns.csv')

df.dropna(inplace=True,axis=1)

df['outcome'] = df['outcome'].apply(lambda x: x.rstrip('.'))


In [18]:
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.0,1.0,0.0,0.00,9,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.0,1.0,0.0,0.00,19,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.0,0.0,1.0,0.0,0.00,29,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.0,1.0,0.0,0.00,39,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.0,1.0,0.0,0.00,49,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,5,0.00,0.00,0.0,0.0,1.0,0.0,0.40,86,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal
494017,0,tcp,http,SF,282,2286,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.0,0.0,1.0,0.0,0.00,6,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal
494018,0,tcp,http,SF,203,1200,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,18,0.17,0.11,0.0,0.0,1.0,0.0,0.17,16,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal
494019,0,tcp,http,SF,291,1200,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,12,0.00,0.00,0.0,0.0,1.0,0.0,0.17,26,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal


## Analyzing a Dataset

In [19]:
def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,\
                int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [20]:
df.groupby('outcome')['outcome'].count()

outcome
back                 2203
buffer_overflow        30
ftp_write               8
guess_passwd           53
imap                   12
ipsweep              1247
land                   21
loadmodule              9
multihop                7
neptune            107201
nmap                  231
normal              97278
perl                    3
phf                     4
pod                   264
portsweep            1040
rootkit                10
satan                1589
smurf              280790
spy                     2
teardrop              979
warezclient          1020
warezmaster            20
Name: outcome, dtype: int64

In [21]:
analyze(df)


494021 rows
** duration:2495 (0%)
** protocol_type:[icmp:57.41%,tcp:38.47%,udp:4.12%]
** service:[ecr_i:56.96%,private:22.45%,http:13.01%,smtp:1.97%,other:1.46%,domain_u:1.19%,ftp_data:0.96%,eco_i:0.33%,ftp:0.16%,finger:0.14%,urp_i:0.11%,telnet:0.1%,ntp_u:0.08%,auth:0.07%,pop_3:0.04%,time:0.03%,csnet_ns:0.03%,remote_job:0.02%,gopher:0.02%,imap4:0.02%,discard:0.02%,domain:0.02%,iso_tsap:0.02%,systat:0.02%,shell:0.02%,echo:0.02%,rje:0.02%,whois:0.02%,sql_net:0.02%,printer:0.02%,nntp:0.02%,courier:0.02%,sunrpc:0.02%,netbios_ssn:0.02%,mtp:0.02%,vmnet:0.02%,uucp_path:0.02%,uucp:0.02%,klogin:0.02%,bgp:0.02%,ssh:0.02%,supdup:0.02%,nnsp:0.02%,login:0.02%,hostnames:0.02%,efs:0.02%,daytime:0.02%,link:0.02%,netbios_ns:0.02%,pop_2:0.02%,ldap:0.02%,netbios_dgm:0.02%,exec:0.02%,http_443:0.02%,kshell:0.02%,name:0.02%,ctf:0.02%,netstat:0.02%,Z39_50:0.02%,IRC:0.01%,urh_i:0.0%,X11:0.0%,tim_i:0.0%,pm_dump:0.0%,tftp_u:0.0%,red_i:0.0%]
** flag:[SF:76.6%,S0:17.61%,REJ:5.44%,RSTR:0.18%,RSTO:0.12%,SH:0.02%,S

## Encode the feature vector

In [22]:
df.loc[df['protocol_type']=='tcp', 'protocol_type'] =1
df.loc[df['protocol_type']=='udp', 'protocol_type'] =2
df.loc[df['protocol_type']=='icmp', 'protocol_type']=3

In [23]:
df.loc[df['flag']== 'SF', 'flag'] = 1
df.loc[df['flag']== 'S0', 'flag'] = 2
df.loc[df['flag']== 'REJ', 'flag'] = 3
df.loc[df['flag']== 'RSTR', 'flag'] = 4
df.loc[df['flag']== 'SH', 'flag'] = 5
df.loc[df['flag']== 'RSTO', 'flag'] = 6
df.loc[df['flag']== 'S1', 'flag'] = 7
df.loc[df['flag']== 'RSTOS0', 'flag'] = 8
df.loc[df['flag']== 'S3', 'flag'] = 9
df.loc[df['flag']== 'S2', 'flag'] = 10
df.loc[df['flag']== 'OTH', 'flag'] = 11

In [24]:
df.groupby('outcome')['outcome'].count()

outcome
back                 2203
buffer_overflow        30
ftp_write               8
guess_passwd           53
imap                   12
ipsweep              1247
land                   21
loadmodule              9
multihop                7
neptune            107201
nmap                  231
normal              97278
perl                    3
phf                     4
pod                   264
portsweep            1040
rootkit                10
satan                1589
smurf              280790
spy                     2
teardrop              979
warezclient          1020
warezmaster            20
Name: outcome, dtype: int64

In [25]:
x = df[['protocol_type','flag','src_bytes','dst_bytes','hot','count','srv_count','same_srv_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_rerror_rate']]
y = df['outcome']
x

,protocol_type,flag,src_bytes,dst_bytes,hot,count,srv_count,same_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_rerror_rate
0,1,1,181,5450,0,8,8,1.0,9,9,1.0,0.0,0.11,0.0
1,1,1,239,486,0,8,8,1.0,19,19,1.0,0.0,0.05,0.0
2,1,1,235,1337,0,8,8,1.0,29,29,1.0,0.0,0.03,0.0
3,1,1,219,1337,0,6,6,1.0,39,39,1.0,0.0,0.03,0.0
4,1,1,217,2032,0,6,6,1.0,49,49,1.0,0.0,0.02,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,1,1,310,1881,0,4,5,1.0,86,255,1.0,0.0,0.01,0.0
494017,1,1,282,2286,0,6,6,1.0,6,255,1.0,0.0,0.17,0.0
494018,1,1,203,1200,0,6,18,1.0,16,255,1.0,0.0,0.06,0.0
494019,1,1,291,1200,0,6,12,1.0,26,255,1.0,0.0,0.04,0.0


In [27]:
y

0         normal
1         normal
2         normal
3         normal
4         normal
           ...  
494016    normal
494017    normal
494018    normal
494019    normal
494020    normal
Name: outcome, Length: 494021, dtype: object

In [28]:
oversample = SMOTE(k_neighbors=1)
x,y = oversample.fit_resample(x,y)
print('Classes and number of values in trainset after SMOTE:',Counter(y))

Classes and number of values in trainset after SMOTE: Counter({'normal': 280790, 'buffer_overflow': 280790, 'loadmodule': 280790, 'perl': 280790, 'neptune': 280790, 'smurf': 280790, 'guess_passwd': 280790, 'pod': 280790, 'teardrop': 280790, 'portsweep': 280790, 'ipsweep': 280790, 'land': 280790, 'ftp_write': 280790, 'back': 280790, 'imap': 280790, 'satan': 280790, 'phf': 280790, 'nmap': 280790, 'multihop': 280790, 'warezmaster': 280790, 'warezclient': 280790, 'spy': 280790, 'rootkit': 280790})


In [29]:
dummies = pd.get_dummies(df['outcome']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

y_df = pd.DataFrame(y, columns=outcomes)
y_df

,back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
494017,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
494018,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
494019,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


## Train the Neural Network

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
x_test = np.asarray(x_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor],verbose=2,epochs=25)

ValueError: Found input variables with inconsistent numbers of samples: [6458170, 494021]

In [ ]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

22815/22815 [==============================] - 15s 653us/step
Validation score: 0.8547162264709186


In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

y_test = np.argmax(y_test, axis=1)

report = classification_report(y_test, y_pred)
print(report)

22815/22815 [==============================] - 14s 618us/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     55931
           1       0.93      0.98      0.96     56195
           2       0.94      0.92      0.93     56063
           3       0.99      1.00      0.99     56221
           4       0.92      0.94      0.93     56061
           5       0.99      0.84      0.91     56133
           6       0.96      0.59      0.73     56207
           7       0.40      1.00      0.57     56128
           8       1.00      0.99      0.99     56283
           9       0.98      0.99      0.99     56369
          10       1.00      1.00      1.00     56141
          11       1.00      0.99      0.99     56259
          12       0.88      0.87      0.87     56063

    accuracy                           0.85    730054
   macro avg       0.85      0.85      0.84    730054
weighted avg       0.85      0.85      0.84    730054



In [ ]:
save_model(model, 'kddDnnAllatck.h5')

with open('outcomesAllatck.pkl', 'wb') as f:
    pickle.dump(outcomes, f)
# # loaded_model = load_model('kddDnn.h5')

In [ ]:
# import tensorflow as tf
# print(tf.__version__)

# import keras
# print(keras.__version__)

# import sys
# print(sys.version)